In [91]:
import re
import camelot
import os
import csv
import pandas as pd
from pathlib import Path



In [92]:
pdf_folder = '../../../tariefkaarten/TOTALENERGIES/PIXEL'
pdf_files = [str(file) for file in Path(pdf_folder).glob("*.pdf")]
csv_file = "pixel.csv"
file_exists = os.path.isfile(csv_file)

In [93]:
header = ["DateKey", "ContractKey", "SingleMeterFixed", "DualMeterDayFixed",
"DualMeterNightFixed", "ExclusiveNightMeterFixed", "SingleMeterVariableMeterFactor",
"SingleMeterVariableBalancingCost", "DualMeterDayVariableMeterFactor",
"DualMeterDayVariableBalancingCost", "DualMeterNightVariableMeterFactor",
"DualMeterNightVariableBalancingCost", "ExclusiveNightMeterVariableMeterFactor",
"ExclusiveNightMeterVariableBalancingCost", "DynamicMeterCost", "DynamicBalancingCost",
"SingleMeterInjectionMeterFactor", "SingleMeterInjectionBalancingCost",
"DualMeterDayInjectionMeterFactor", "DualMeterDayInjectionBalancingCost",
"DualMeterNightInjectionMeterFactor", "DualMeterNightInjectionBalancingCost",
"AdministrativeCosts", "GreenElectricity", "WKK"]

df_pixel = pd.DataFrame(columns=header)

df_pixel.head()

,DateKey,ContractKey,SingleMeterFixed,DualMeterDayFixed,DualMeterNightFixed,ExclusiveNightMeterFixed,SingleMeterVariableMeterFactor,SingleMeterVariableBalancingCost,DualMeterDayVariableMeterFactor,DualMeterDayVariableBalancingCost,...,DynamicBalancingCost,SingleMeterInjectionMeterFactor,SingleMeterInjectionBalancingCost,DualMeterDayInjectionMeterFactor,DualMeterDayInjectionBalancingCost,DualMeterNightInjectionMeterFactor,DualMeterNightInjectionBalancingCost,AdministrativeCosts,GreenElectricity,WKK


In [94]:
for file_path in pdf_files:
    try:
        print(f"Processing {file_path}")
        tables = camelot.read_pdf(file_path, pages='1')
        
        match = re.search(r'(\d{4})-(\d{2})', os.path.basename(file_path))
        date_key = f"{match.group(1)}{match.group(2)}01" if match else "Unknown"
        year = int(match.group(1)) if match else 0
        month = int(match.group(2)) if match else 0
        
        # Check if this is a newer PDF format (2023-04 and later or 2023-09)
        use_new_indices = False
        if (year > 2023) or (year == 2023 and (month >= 4 or month == 9)):
            use_new_indices = True
        
        # Use the appropriate table indices based on the PDF date
        if use_new_indices:
            # For newer PDFs (2023-04 and later)
            verbruik_table_idx = 0
            wkk_table_idx = 1
            vaste_vergoeding_idx = 2
        else:
            # For older PDFs
            verbruik_table_idx = 1
            wkk_table_idx = 2
            vaste_vergoeding_idx = 3
        
        WKK = tables[wkk_table_idx].df
        vasteVergoeding = tables[vaste_vergoeding_idx].df
        
        row4 = tables[verbruik_table_idx].df.iloc[4]

        col0_text = row4[0]
        col0_parts = col0_text.split('\\n') if '\\n' in col0_text else col0_text.split('\n')
        one = col0_parts[0].split('*')[0].strip()
        two = col0_parts[0].split('+')[1].strip() if '+' in col0_parts[0] else (col0_parts[1].split('+')[1].strip() if len(col0_parts) > 1 and '+' in col0_parts[1] else "")

        col1_text = row4[1]
        col1_parts = col1_text.split('\\n') if '\\n' in col1_text else col1_text.split('\n')
        three = col1_parts[0].split('*')[0].strip()
        four = col1_parts[1].split('*')[0].strip() if len(col1_parts) > 1 else ""
        five = col1_parts[2].split('+')[1].strip()
        six = col1_parts[3].split('+')[1].strip()

        col2_text = row4[2]
        col2_parts = col2_text.split('\\n') if '\\n' in col2_text else col2_text.split('\n')
        seven = col2_parts[0].split('*')[0].strip()
        eight = col2_parts[0].split('+')[1].strip() if '+' in col2_parts[0] else (col2_parts[1].split('+')[1].strip() if len(col2_parts) > 1 and '+' in col2_parts[1] else "")

        values = [one, two, three, four, five, six, seven, eight]
        values = [v.replace(',', '.') if isinstance(v, str) else v for v in values]
        
        new_row = {
                "DateKey": date_key,
                "ContractKey": "12",
                "SingleMeterVariableMeterFactor": values[0],
                "SingleMeterVariableBalancingCost": values[1],
                "DualMeterDayVariableMeterFactor": values[2],
                "DualMeterDayVariableBalancingCost": values[3],
                "DualMeterNightVariableMeterFactor": values[4],
                "DualMeterNightVariableBalancingCost": values[5],
                "ExclusiveNightMeterVariableMeterFactor": values[6],
                "ExclusiveNightMeterVariableBalancingCost": values[7],
                " AdministrativeCosts": vasteVergoeding.at[1, 0],
                "GreenElectricity": WKK.at[1, 0],
            }
        
        new_row_df = pd.DataFrame([new_row])
        df_pixel = pd.concat([df_pixel, new_row_df], ignore_index=True)
        print(f"Successfully processed {file_path}")

    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")

Processing ..\..\..\tariefkaarten\TOTALENERGIES\PIXEL\totalenergies-pixel-2023-01.pdf
Successfully processed ..\..\..\tariefkaarten\TOTALENERGIES\PIXEL\totalenergies-pixel-2023-01.pdf
Processing ..\..\..\tariefkaarten\TOTALENERGIES\PIXEL\totalenergies-pixel-2023-02.pdf
Successfully processed ..\..\..\tariefkaarten\TOTALENERGIES\PIXEL\totalenergies-pixel-2023-02.pdf
Processing ..\..\..\tariefkaarten\TOTALENERGIES\PIXEL\totalenergies-pixel-2023-03.pdf
Successfully processed ..\..\..\tariefkaarten\TOTALENERGIES\PIXEL\totalenergies-pixel-2023-03.pdf
Processing ..\..\..\tariefkaarten\TOTALENERGIES\PIXEL\totalenergies-pixel-2023-04.pdf
Successfully processed ..\..\..\tariefkaarten\TOTALENERGIES\PIXEL\totalenergies-pixel-2023-04.pdf
Processing ..\..\..\tariefkaarten\TOTALENERGIES\PIXEL\totalenergies-pixel-2023-05.pdf
Successfully processed ..\..\..\tariefkaarten\TOTALENERGIES\PIXEL\totalenergies-pixel-2023-05.pdf
Processing ..\..\..\tariefkaarten\TOTALENERGIES\PIXEL\totalenergies-pixel-2023-0

In [95]:
import pandas as pd
import re
import os

manual_values = {
    "20230901": {
        "SingleMeterVariableMeterFactor": "0.1071",
        "SingleMeterVariableBalancingCost": "2.527",
        "DualMeterDayVariableMeterFactor": "0.1213",
        "DualMeterDayVariableBalancingCost": "2.527",
        "DualMeterNightVariableMeterFactor": "0.0935",
        "DualMeterNightVariableBalancingCost": "2.527",
        "ExclusiveNightMeterVariableMeterFactor": "0.0971",
        "ExclusiveNightMeterVariableBalancingCost": "2.365",
        "AdministrativeCosts": "35.0",
        "GreenElectricity": "2.2190"
    },
    # August 2024 PDF
    "20240801": {
        "SingleMeterVariableMeterFactor": "0.1071",
        "SingleMeterVariableBalancingCost": "1.77",
        "DualMeterDayVariableMeterFactor": "0.1177",
        "DualMeterDayVariableBalancingCost": "1.77",
        "DualMeterNightVariableMeterFactor": "0.0975",
        "DualMeterNightVariableBalancingCost": "1.77",
        "ExclusiveNightMeterVariableMeterFactor": "0.1023",
        "ExclusiveNightMeterVariableBalancingCost": "1.59",
        "SingleMeterInjectionMeterFactor": "0.054",
        "SingleMeterInjectionBalancingCost": "-0.7",
        "DualMeterDayInjectionMeterFactor": "0.054",
        "DualMeterDayInjectionBalancingCost": "-0.7",
        "DualMeterNightInjectionMeterFactor": "0.054",
        "DualMeterNightInjectionBalancingCost": "-0.7",
        "AdministrativeCosts": "55.00",
        "GreenElectricity": "1.58"
        
    },
    "20240901": {
        "SingleMeterVariableMeterFactor": "0.1071",
        "SingleMeterVariableBalancingCost": "1.73",
        "DualMeterDayVariableMeterFactor": "0.1183",
        "DualMeterDayVariableBalancingCost": "1.73",
        "DualMeterNightVariableMeterFactor": "0.097",
        "DualMeterNightVariableBalancingCost": "1.73",
        "ExclusiveNightMeterVariableMeterFactor": "0.102",
        "ExclusiveNightMeterVariableBalancingCost": "1.56",
        "SingleMeterInjectionMeterFactor": "0.054",
        "SingleMeterInjectionBalancingCost": "-0.7",
        "DualMeterDayInjectionMeterFactor": "0.054",
        "DualMeterDayInjectionBalancingCost": "-0.7",
        "DualMeterNightInjectionMeterFactor": "0.054",
        "DualMeterNightInjectionBalancingCost": "-0.7",
        "AdministrativeCosts": "55.00",
        "GreenElectricity": "1.58"
    },
    # October 2024 PDF
    "20241001": {
        "SingleMeterVariableMeterFactor": "0.1088",
        "SingleMeterVariableBalancingCost": "1.76",
        "DualMeterDayVariableMeterFactor": "0.1199",
        "DualMeterDayVariableBalancingCost": "1.76",
        "DualMeterNightVariableMeterFactor": "0.0987",
        "DualMeterNightVariableBalancingCost": "1.76",
        "ExclusiveNightMeterVariableMeterFactor": "0.1047",
        "ExclusiveNightMeterVariableBalancingCost": "1.55",
        "SingleMeterInjectionMeterFactor": "0.038",
        "SingleMeterInjectionBalancingCost": "-0.625",
        "DualMeterDayInjectionMeterFactor": "0.038",
        "DualMeterDayInjectionBalancingCost": "-0.625",
        "DualMeterNightInjectionMeterFactor": "0.038",
        "DualMeterNightInjectionBalancingCost": "-0.625",
        "AdministrativeCosts": "55.00",
        "GreenElectricity": "1.58"
    },
    # November 2024 PDF
    "20241101": {
        "SingleMeterVariableMeterFactor": "0.1093",
        "SingleMeterVariableBalancingCost": "1.73",
        "DualMeterDayVariableMeterFactor": "0.1205",
        "DualMeterDayVariableBalancingCost": "1.73",
        "DualMeterNightVariableMeterFactor": "0.0993",
        "DualMeterNightVariableBalancingCost": "1.73",
        "ExclusiveNightMeterVariableMeterFactor": "0.1051",
        "ExclusiveNightMeterVariableBalancingCost": "1.54",
        "ExclusiveNightMeterVariableBalancingCost": "1.55",
        "SingleMeterInjectionMeterFactor": "0.0376",
        "SingleMeterInjectionBalancingCost": "-0.625",
        "DualMeterDayInjectionMeterFactor": "0.0376",
        "DualMeterDayInjectionBalancingCost": "-0.625",
        "DualMeterNightInjectionMeterFactor": "0.0376",
        "DualMeterNightInjectionBalancingCost": "-0.625",
        "AdministrativeCosts": "55.00",
        "GreenElectricity": "1.58"
    },
    # December 2024 PDF
    "20241201": {
        "SingleMeterVariableMeterFactor": "0.1092",
        "SingleMeterVariableBalancingCost": "1.68",
        "DualMeterDayVariableMeterFactor": "0.1193",
        "DualMeterDayVariableBalancingCost": "1.68",
        "DualMeterNightVariableMeterFactor": "0.1001",
        "DualMeterNightVariableBalancingCost": "1.68",
        "ExclusiveNightMeterVariableMeterFactor": "0.1061",
        "ExclusiveNightMeterVariableBalancingCost": "1.46",
        "SingleMeterInjectionMeterFactor": "0.0376",
        "SingleMeterInjectionBalancingCost": "-0.625",
        "DualMeterDayInjectionMeterFactor": "0.0376",
        "DualMeterDayInjectionBalancingCost": "-0.625",
        "DualMeterNightInjectionMeterFactor": "0.0376",
        "DualMeterNightInjectionBalancingCost": "-0.625",
        "AdministrativeCosts": "55.00",
        "GreenElectricity": "1.58"
    },
    # January 2025 PDF
    "20250101": {
        "SingleMeterVariableMeterFactor": "0.1084",
        "SingleMeterVariableBalancingCost": "1.78",
        "DualMeterDayVariableMeterFactor": "0.1176",
        "DualMeterDayVariableBalancingCost": "1.78",
        "DualMeterNightVariableMeterFactor": "0.1002",
        "DualMeterNightVariableBalancingCost": "1.78",
        "ExclusiveNightMeterVariableMeterFactor": "0.1068",
        "ExclusiveNightMeterVariableBalancingCost": "1.55",
        "SingleMeterInjectionMeterFactor": "0.0376",
        "SingleMeterInjectionBalancingCost": "-0.625",
        "DualMeterDayInjectionMeterFactor": "0.0376",
        "DualMeterDayInjectionBalancingCost": "-0.625",
        "DualMeterNightInjectionMeterFactor": "0.0376",
        "DualMeterNightInjectionBalancingCost": "-0.625",
        "AdministrativeCosts": "55.00",
        "GreenElectricity": "1.58"
    },
    # February 2025 PDF
    "20250201": {
        "SingleMeterVariableMeterFactor": "0.1084",
        "SingleMeterVariableBalancingCost": "1.78",
        "DualMeterDayVariableMeterFactor": "0.1176",
        "DualMeterDayVariableBalancingCost": "1.78",
        "DualMeterNightVariableMeterFactor": "0.1002",
        "DualMeterNightVariableBalancingCost": "1.78",
        "ExclusiveNightMeterVariableMeterFactor": "0.1068",
        "ExclusiveNightMeterVariableBalancingCost": "1.55",
        "SingleMeterInjectionMeterFactor": "0.0376",
        "SingleMeterInjectionBalancingCost": "-0.625",
        "DualMeterDayInjectionMeterFactor": "0.0376",
        "DualMeterDayInjectionBalancingCost": "-0.625",
        "DualMeterNightInjectionMeterFactor": "0.0376",
        "DualMeterNightInjectionBalancingCost": "-0.625",
        "AdministrativeCosts": "55.00",
        "GreenElectricity": "1.58"
    }, 
    # March 2025 PDF
    "20250301": {
        "SingleMeterVariableMeterFactor": "0.1085",
        "SingleMeterVariableBalancingCost": "1.74",
        "DualMeterDayVariableMeterFactor": "0.1219",
        "DualMeterDayVariableBalancingCost": "1.74",
        "DualMeterNightVariableMeterFactor": "0.0965",
        "DualMeterNightVariableBalancingCost": "1.74",
        "ExclusiveNightMeterVariableMeterFactor": "0.1009",
        "ExclusiveNightMeterVariableBalancingCost": "1.51",
        "SingleMeterInjectionMeterFactor": "0.054",
        "SingleMeterInjectionBalancingCost": "-0.7",
        "DualMeterDayInjectionMeterFactor": "0.054",
        "DualMeterDayInjectionBalancingCost": "-0.7",
        "DualMeterNightInjectionMeterFactor": "0.054",
        "DualMeterNightInjectionBalancingCost": "-0.7",
        "AdministrativeCosts": "55.00",
        "GreenElectricity": "1.58"
    }
}

# Function to process PDFs that failed with camelot
def process_failed_pdf(file_path, df_pixel):
    try:
        print(f"Processing {file_path} manually")
        
        # Extract date from filename
        match = re.search(r'(\d{4})-(\d{2})', os.path.basename(file_path))
        if not match:
            raise ValueError(f"Could not extract date from filename: {file_path}")
            
        year = int(match.group(1))
        month = int(match.group(2))
        date_key = f"{year}{month:02d}01"
        
        # Check if we have manual values for this PDF
        if date_key not in manual_values:
            raise ValueError(f"No manual values found for date: {date_key}")
        
        values = manual_values[date_key]
        
        # Create a new row with the extracted values
        new_row = {
            "DateKey": date_key,
            "ContractKey": "12",
            "SingleMeterVariableMeterFactor": values["SingleMeterVariableMeterFactor"],
            "SingleMeterVariableBalancingCost": values["SingleMeterVariableBalancingCost"],
            "DualMeterDayVariableMeterFactor": values["DualMeterDayVariableMeterFactor"],
            "DualMeterDayVariableBalancingCost": values["DualMeterDayVariableBalancingCost"],
            "DualMeterNightVariableMeterFactor": values["DualMeterNightVariableMeterFactor"],
            "DualMeterNightVariableBalancingCost": values["DualMeterNightVariableBalancingCost"],
            "ExclusiveNightMeterVariableMeterFactor": values["ExclusiveNightMeterVariableMeterFactor"],
            "ExclusiveNightMeterVariableBalancingCost": values["ExclusiveNightMeterVariableBalancingCost"],
            "SingleMeterInjectionMeterFactor": values["SingleMeterInjectionMeterFactor"],
            "SingleMeterInjectionBalancingCost": values["SingleMeterInjectionBalancingCost"],
            "DualMeterDayInjectionMeterFactor": values["DualMeterDayInjectionMeterFactor"],
            "DualMeterDayInjectionBalancingCost": values["DualMeterDayInjectionBalancingCost"],
            "DualMeterNightInjectionMeterFactor": values["DualMeterNightInjectionMeterFactor"],
            "DualMeterNightInjectionBalancingCost": values["DualMeterNightInjectionBalancingCost"],
            " AdministrativeCosts": values["AdministrativeCosts"],
            "GreenElectricity": values["GreenElectricity"],
        }
        
        # Add the new row to the dataframe
        new_row_df = pd.DataFrame([new_row])
        df_pixel = pd.concat([df_pixel, new_row_df], ignore_index=True)
        print(f"Successfully processed {file_path}")
        
        return df_pixel
        
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return df_pixel

# Example usage
problematic_files = [
    "totalenergies-pixel-2023-09.pdf",
    "totalenergies-pixel-2024-08.pdf",
    "totalenergies-pixel-2024-09.pdf",
    "totalenergies-pixel-2024-10.pdf",
    "totalenergies-pixel-2024-11.pdf",
    "totalenergies-pixel-2024-12.pdf",
    "totalenergies-pixel-2025-01.pdf",
    "totalenergies-pixel-2025-02.pdf",
    "totalenergies-pixel-2025-03.pdf"
]

# Process each problematic file
for file_path in problematic_files:
    df_pixel = process_failed_pdf(file_path, df_pixel)

# To integrate with the main processing loop, modify your existing script:
"""
for file_path in pdf_files:
    try:
        # First try to process normally with camelot
        print(f"Processing {file_path}")
        tables = camelot.read_pdf(file_path, pages='1')
        
        # Check if this is a problematic PDF that needs manual handling
        if any(problem_file in file_path for problem_file in problematic_files):
            # Use manual processing instead
            df_pixel = process_failed_pdf(file_path, df_pixel)
            continue
            
        # Rest of your existing processing code...
        match = re.search(r'(\d{4})-(\d{2})', os.path.basename(file_path))
        date_key = f"{match.group(1)}{match.group(2)}01" if match else "Unknown"
        # ... and so on
        
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
"""

Processing totalenergies-pixel-2023-09.pdf manually
Error processing totalenergies-pixel-2023-09.pdf: 'SingleMeterInjectionMeterFactor'
Processing totalenergies-pixel-2024-08.pdf manually
Successfully processed totalenergies-pixel-2024-08.pdf
Processing totalenergies-pixel-2024-09.pdf manually
Successfully processed totalenergies-pixel-2024-09.pdf
Processing totalenergies-pixel-2024-10.pdf manually
Successfully processed totalenergies-pixel-2024-10.pdf
Processing totalenergies-pixel-2024-11.pdf manually
Successfully processed totalenergies-pixel-2024-11.pdf
Processing totalenergies-pixel-2024-12.pdf manually
Successfully processed totalenergies-pixel-2024-12.pdf
Processing totalenergies-pixel-2025-01.pdf manually
Successfully processed totalenergies-pixel-2025-01.pdf
Processing totalenergies-pixel-2025-02.pdf manually
Successfully processed totalenergies-pixel-2025-02.pdf
Processing totalenergies-pixel-2025-03.pdf manually
Successfully processed totalenergies-pixel-2025-03.pdf


<>:244: SyntaxWarning: invalid escape sequence '\d'
<>:244: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sfsan\AppData\Local\Temp\ipykernel_28244\1815455633.py:244: SyntaxWarning: invalid escape sequence '\d'
  """


'\nfor file_path in pdf_files:\n    try:\n        # First try to process normally with camelot\n        print(f"Processing {file_path}")\n        tables = camelot.read_pdf(file_path, pages=\'1\')\n        \n        # Check if this is a problematic PDF that needs manual handling\n        if any(problem_file in file_path for problem_file in problematic_files):\n            # Use manual processing instead\n            df_pixel = process_failed_pdf(file_path, df_pixel)\n            continue\n            \n        # Rest of your existing processing code...\n        match = re.search(r\'(\\d{4})-(\\d{2})\', os.path.basename(file_path))\n        date_key = f"{match.group(1)}{match.group(2)}01" if match else "Unknown"\n        # ... and so on\n        \n    except Exception as e:\n        print(f"Error processing {file_path}: {str(e)}")\n'

In [96]:
df_pixel.head(30)

,DateKey,ContractKey,SingleMeterFixed,DualMeterDayFixed,DualMeterNightFixed,ExclusiveNightMeterFixed,SingleMeterVariableMeterFactor,SingleMeterVariableBalancingCost,DualMeterDayVariableMeterFactor,DualMeterDayVariableBalancingCost,...,SingleMeterInjectionMeterFactor,SingleMeterInjectionBalancingCost,DualMeterDayInjectionMeterFactor,DualMeterDayInjectionBalancingCost,DualMeterNightInjectionMeterFactor,DualMeterNightInjectionBalancingCost,AdministrativeCosts,GreenElectricity,WKK,AdministrativeCosts
0,20230101,12,NaN,NaN,NaN,NaN,0.1065,3.010,0.128,0.088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,5091",NaN,"25,0"
1,20230201,12,NaN,NaN,NaN,NaN,0.1065,3.010,0.128,0.088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,5091",NaN,"25,0"
2,20230301,12,NaN,NaN,NaN,NaN,0.1065,2.51,0.128,0.088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,5091",NaN,"25,0"
3,20230401,12,NaN,NaN,NaN,NaN,0.1065,2.51,0.128,0.088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,1557",NaN,"25,0"
4,20230501,12,NaN,NaN,NaN,NaN,0.1065,2.51,0.128,0.088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,4608",NaN,"25,0"
5,20230601,12,NaN,NaN,NaN,NaN,0.1065,2.51,0.1280,0.0880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,1557",NaN,"25,0"
6,20230701,12,NaN,NaN,NaN,NaN,0.1065,2.51,0.1280,0.0880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,4608",NaN,"25,0"
7,20230801,12,NaN,NaN,NaN,NaN,0.1065,2.51,0.1280,0.0880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,2190",NaN,"30,0"
8,20231001,12,NaN,NaN,NaN,NaN,0.1071,2.53,0.1215,0.0935,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,2190",NaN,"35,0"
9,20231101,12,NaN,NaN,NaN,NaN,0.1071,2.44,0.121,0.094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,2238",NaN,"35,0"


In [97]:
from decimal import Decimal, InvalidOperation

# Zet in de juiste datatypes en format voor inladen in de database

df_pixel['DateKey'] = df_pixel['DateKey'].astype(int)
df_pixel['ContractKey'] = df_pixel['ContractKey'].astype(int)

for col in df_pixel.columns:
    if col not in ['DateKey', 'ContractKey']:
        df_pixel[col] = df_pixel[col].apply(
            lambda x: '{:.6f}'.format(float(str(x).replace(',', '.'))) if pd.notna(x) and (str(x).replace(',', '.').replace('.', '', 1).isdigit() or (str(x).startswith('-') and str(x).replace('-', '', 1).replace(',', '.').replace('.', '', 1).isdigit())) else ''
        )

csv_path = 'pixel.csv'
df_pixel.to_csv(
    csv_path,
    index=False,
    sep=',',
    decimal='.',
    quoting=3,
    escapechar='\\',
    date_format='%Y%m%d'
)

print(f"CSV file created at: {csv_path}")
df_pixel.head(20)


CSV file created at: pixel.csv


,DateKey,ContractKey,SingleMeterFixed,DualMeterDayFixed,DualMeterNightFixed,ExclusiveNightMeterFixed,SingleMeterVariableMeterFactor,SingleMeterVariableBalancingCost,DualMeterDayVariableMeterFactor,DualMeterDayVariableBalancingCost,...,SingleMeterInjectionMeterFactor,SingleMeterInjectionBalancingCost,DualMeterDayInjectionMeterFactor,DualMeterDayInjectionBalancingCost,DualMeterNightInjectionMeterFactor,DualMeterNightInjectionBalancingCost,AdministrativeCosts,GreenElectricity,WKK,AdministrativeCosts
0,20230101,12,,,,,0.106500,3.010000,0.128000,0.088000,...,,,,,,,,2.509100,,25.000000
1,20230201,12,,,,,0.106500,3.010000,0.128000,0.088000,...,,,,,,,,2.509100,,25.000000
2,20230301,12,,,,,0.106500,2.510000,0.128000,0.088000,...,,,,,,,,2.509100,,25.000000
3,20230401,12,,,,,0.106500,2.510000,0.128000,0.088000,...,,,,,,,,2.155700,,25.000000
4,20230501,12,,,,,0.106500,2.510000,0.128000,0.088000,...,,,,,,,,2.460800,,25.000000
5,20230601,12,,,,,0.106500,2.510000,0.128000,0.088000,...,,,,,,,,2.155700,,25.000000
6,20230701,12,,,,,0.106500,2.510000,0.128000,0.088000,...,,,,,,,,2.460800,,25.000000
7,20230801,12,,,,,0.106500,2.510000,0.128000,0.088000,...,,,,,,,,2.219000,,30.000000
8,20231001,12,,,,,0.107100,2.530000,0.121500,0.093500,...,,,,,,,,2.219000,,35.000000
9,20231101,12,,,,,0.107100,2.440000,0.121000,0.094000,...,,,,,,,,2.223800,,35.000000
